In [ ]:
import pandas as pd
import world_bank_data as wb
pd.set_option('display.max_rows', 6)
# https://pypi.org/project/world-bank-data/

In [2]:
wb.get_topics()

,value,sourceNote
id,,
1,Agriculture & Rural Development,For the 70 percent of the world's poor who liv...
2,Aid Effectiveness,Aid effectiveness is the impact that aid has i...
3,Economy & Growth,Economic growth is central to economic develop...
...,...,...
19,Climate Change,Climate change is expected to hit developing c...
20,External Debt,Debt statistics provide a detailed picture of ...
21,Trade,Trade is a key means to fight poverty and achi...


In [3]:
wb.get_sources()

,lastupdated,name,code,description,url,dataavailability,metadataavailability,concepts
id,,,,,,,,
1,2021-08-18,Doing Business,DBS,,,Y,Y,3
2,2024-10-24,World Development Indicators,WDI,,,Y,Y,3
3,2024-11-05,Worldwide Governance Indicators,WGI,,,Y,Y,3
...,...,...,...,...,...,...,...,...
89,2022-12-21,Identification for Development (ID4D) Data,ID4,,,Y,Y,3
90,2024-08-04,ICP 2021,IC2,,,Y,Y,4
91,2024-08-19,PEFA_CRPFM,CRP,,,Y,Y,4


In [4]:
wb.get_countries()

,iso2Code,name,region,adminregion,incomeLevel,lendingType,capitalCity,longitude,latitude
id,,,,,,,,,
ABW,AW,Aruba,Latin America & Caribbean,,High income,Not classified,Oranjestad,-70.0167,12.5167
AFE,ZH,Africa Eastern and Southern,Aggregates,,Aggregates,Aggregates,,NaN,NaN
AFG,AF,Afghanistan,South Asia,South Asia,Low income,IDA,Kabul,69.1761,34.5228
...,...,...,...,...,...,...,...,...,...
ZAF,ZA,South Africa,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Upper middle income,IBRD,Pretoria,28.1871,-25.7460
ZMB,ZM,Zambia,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,IDA,Lusaka,28.2937,-15.3982
ZWE,ZW,Zimbabwe,Sub-Saharan Africa,Sub-Saharan Africa (excluding high income),Lower middle income,Blend,Harare,31.0672,-17.8312


In [5]:
wb.get_indicators(topic=3, source=2)  # topic and source id are from get_topics/get_sources

,name,unit,source,sourceNote,sourceOrganization,topics
id,,,,,,
BG.GSR.NFSV.GD.ZS,Trade in services (% of GDP),,World Development Indicators,Trade in services is the sum of service export...,"International Monetary Fund, Balance of Paymen...","Economy & Growth,Private Sector,Trade"
BM.GSR.CMCP.ZS,"Communications, computer, etc. (% of service i...",,World Development Indicators,"Communications, computer, information, and oth...","International Monetary Fund, Balance of Paymen...","Economy & Growth,Trade"
BM.GSR.FCTY.CD,"Primary income payments (BoP, current US$)",,World Development Indicators,Primary income payments refer to employee comp...,"International Monetary Fund, Balance of Paymen...",Economy & Growth
...,...,...,...,...,...,...
PA.NUS.PPP,"PPP conversion factor, GDP (LCU per internatio...",,World Development Indicators,Purchasing power parity (PPP) conversion facto...,"International Comparison Program, World Bank |...",Economy & Growth
PA.NUS.PPPC.RF,Price level ratio of PPP conversion factor (GD...,,World Development Indicators,Price level ratio is the ratio of a purchasing...,"International Comparison Program, World Bank |...",Economy & Growth
PA.NUS.PRVT.PP,"PPP conversion factor, private consumption (LC...",,World Development Indicators,Purchasing power parity (PPP) conversion facto...,"International Comparison Program, World Bank |...",Economy & Growth


In [6]:
wb.get_series('SP.POP.TOTL', mrv=1)

Country                      Series             Year
Africa Eastern and Southern  Population, total  2023    739108306.0
Africa Western and Central   Population, total  2023    502789511.0
Arab World                   Population, total  2023    473272080.0
                                                           ...     
Yemen, Rep.                  Population, total  2023     34449825.0
Zambia                       Population, total  2023     20569737.0
Zimbabwe                     Population, total  2023     16665409.0
Name: SP.POP.TOTL, Length: 266, dtype: float64

In [7]:
wb.get_series('SP.POP.TOTL', date='2016', id_or_value='id', simplify_index=True)

Country
AFE    616377605.0
AFW    419778384.0
ARB    415077960.0
          ...     
YEM     29274002.0
ZMB     16767761.0
ZWE     14452704.0
Name: SP.POP.TOTL, Length: 266, dtype: float64

In [ ]:
time.sleep(5)

driver.find_element(By.XPATH, "//input[contains(@class, 'cosmos-input')]").send_keys("difeliprpo@hotmail.com" )

driver.save_screenshot("screenshot.png")

In [ ]:
driver.find_element(By.XPATH, "//button[contains(@class, 'cosmos-btn')]").click()

driver.save_screenshot("screenshot.png")